In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 18.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.7 MB/s eta 0:00:00


In [4]:
import easyocr

# Load the image file path
image_path = './data/sample/ocr.png'  # Replace with your local path

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'], gpu =  True)

# Perform text detection and recognition
results = reader.readtext(image_path)

# Print the results
for (bbox, text, confidence) in results:
    print(f"Detected text: '{text}' with confidence: {confidence}")

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteDetected text: 'CHE' with confidence: 0.9995971751298354
Detected text: '1-01' with confidence: 0.6182919604570368
Detected text: 'BUR' with confidence: 0.9999307618526357
Detected text: '49.55' with confidence: 0.8905346903352951


/home/thiendc/projects/.thienenv/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


In [9]:
float("50")

50.0

In [ ]:

def is_timestamp(text: str):
    try:
        float(text)
        return text
    except ValueError:
        if ':' in text:
            before , _ = text.split(":")
            try:
                float(before.strip())
                return text
            except:
                return text
        elif ',' in text:
            try:
                float(before.strip())
                return text.replace(",", ":")
            except:
                return text.replace(",", ":")
        else:
            pass

is_timestamp("1-01")

In [52]:
data =  [
    ([[95, 15], [129, 15], [129, 33], [95, 33]], 'CHE', 0.9995971751298354),
    ([[139, 15], [183, 15], [183, 35], [139, 35]], '1-01', 0.6182919604570368),
    ([[191, 15], [223, 15], [223, 33], [191, 33]], 'BUR', 0.9999307618526357),
    ([[99, 39], [139, 39], [139, 55], [99, 55]], '49.55', 0.8905346903352951)
    ]

for item in data:
    values = item[1]
    print(values)

CHE
1-01
BUR
49.55


In [61]:
data

[([[95, 15], [129, 15], [129, 33], [95, 33]], 'CHE', 0.9995971751298354),
 ([[139, 15], [183, 15], [183, 35], [139, 35]], '1-01', 0.6182919604570368),
 ([[191, 15], [223, 15], [223, 33], [191, 33]], 'BUR', 0.9999307618526357),
 ([[99, 39], [139, 39], [139, 55], [99, 55]], '49.55', 0.8905346903352951)]

In [ ]:
from typing import List
def get_height(data: List):
    """ 
        Input:
            text có bbox của easyocr 
            [ [x1, y1], [x2, y2], [x3, y3], [x4, y4] ]
        Output
            y1 values
    """
    bbox = [item[0] for item in data]
    y_values = [point[0][1] for point in bbox]
    # y1 = y_values[0]
    return y_values

def get_item(data: List):

    # Get y1 values in bboxes1
    bbox = [item[0] for item in data]
    y_values = [point[0][1] for point in bbox]
    # get text 
    text = [item[1] for item in data]
    # get confidence score
    conf_score = [item[2] for item in data]
    return y_values, text, conf_score

a,b,c =  get_item(data)
# text, score = get_text(data)

[15, 15, 15, 39]

In [92]:
list(zip(a,b,c, data))

[(15,
  'CHE',
  0.9995971751298354,
  ([[95, 15], [129, 15], [129, 33], [95, 33]], 'CHE', 0.9995971751298354)),
 (15,
  '1-01',
  0.6182919604570368,
  ([[139, 15], [183, 15], [183, 35], [139, 35]], '1-01', 0.6182919604570368)),
 (15,
  'BUR',
  0.9999307618526357,
  ([[191, 15], [223, 15], [223, 33], [191, 33]], 'BUR', 0.9999307618526357)),
 (39,
  '49.55',
  0.8905346903352951,
  ([[99, 39], [139, 39], [139, 55], [99, 55]], '49.55', 0.8905346903352951))]

In [ ]:
# current_y = a[0]
# for i, j, k in zip(a,b,c):
#     if abs(i - current_y) <= 2:
#         current_line.append()

from typing import List

def get_item(data: List):
    # Lấy các tọa độ bounding boxes và thông tin y_values, confidence score và text
    bbox = [item[0] for item in data]
    y_values = [point[0][1] for point in bbox]
    text = [item[1] for item in data]
    conf_score = [item[2] for item in data]
    
    return y_values, conf_score, text

def group_by_lines(data: List, y_threshold: float = 5.0):
    """
    Chia dữ liệu thành các dòng (lines) dựa trên các giá trị y của bounding boxes.
    Dữ liệu có giá trị y gần nhau sẽ được coi là cùng một dòng.
    """
    # Lấy thông tin y_values, confidence score và text
    y_values, conf_scores, texts = get_item(data)
    
    # Sắp xếp các item theo giá trị y (từ thấp đến cao)
    sorted_data = list(zip(y_values, conf_scores, texts, data))
    
    # Khởi tạo danh sách các dòng (lines)
    lines = []
    current_line = []
    current_y = sorted_data[0][0]
    
    # Duyệt qua từng item trong dữ liệu đã sắp xếp và nhóm chúng theo dòng
    for y, _, _, item in sorted_data:
        # Nếu y_value của item này gần với y_value hiện tại, chúng cùng một dòng
        if abs(y - current_y) <= y_threshold:
            current_line.append(item)
        else:
            # Nếu không, tạo một dòng mới
            lines.append(current_line)
            current_line = [item]
            current_y = y
    
    # Đừng quên thêm dòng cuối vào danh sách
    if current_line:
        lines.append(current_line)
    
    return lines

# Dữ liệu
data = [
    ([[95, 15], [129, 15], [129, 33], [95, 33]], 'CHE', 0.9995971751298354),
    ([[139, 15], [183, 15], [183, 35], [139, 35]], '1-01', 0.6182919604570368),
    ([[191, 15], [223, 15], [223, 33], [191, 33]], 'BUR', 0.9999307618526357),
    ([[99, 39], [139, 39], [139, 55], [99, 55]], '49.55', 0.8905346903352951)
]

# Chia dữ liệu thành các dòng
lines = group_by_lines(data)

# In ra các dòng và các item trong từng dòng
for idx, line in enumerate(lines):
    print(f"Line {idx + 1}:")
    for item in line:
        print(f"  Label: {item[1]}, Confidence: {item[2]}, Bounding Box: {item[0]}")


Line 1:
  Label: CHE, Confidence: 0.9995971751298354, Bounding Box: [[95, 15], [129, 15], [129, 33], [95, 33]]
  Label: 1-01, Confidence: 0.6182919604570368, Bounding Box: [[139, 15], [183, 15], [183, 35], [139, 35]]
  Label: BUR, Confidence: 0.9999307618526357, Bounding Box: [[191, 15], [223, 15], [223, 33], [191, 33]]
Line 2:
  Label: 49.55, Confidence: 0.8905346903352951, Bounding Box: [[99, 39], [139, 39], [139, 55], [99, 55]]


In [142]:

def is_timestampt(text: str):
    """
        Function:
            Kiểm tra một giá trị bounding box có phải là timestamp không
            Một bbox được coi là timestamp khi được phân biệt bởi dấu , hoặc : hoặc có dấu .

        Output:
            Một output nếu không phải là timestamp thì sẽ trả về None
    """
    try:
        float(text)
        text = text.replace(".", ":")
        return text
    except ValueError:
        if ':' in text:
            before , _ = text.split(":")
            try:
                float(before.strip())
                return text
            except:
                return text
        elif ',' in text:
            try:
                float(before.strip())
                return text.replace(",", ":")
            except:
                return text.replace(",", ":")
        else:
            pass

for line in lines:
    text = line[0][1]
    if is_timestampt(text):
        current_time = is_timestampt(text)
        print(current_time)
    else:
        print("timestamp is not recognized")
    

timestamp is not recognized
49:55


In [ ]:
for line in lines:
    text1, text2 =  line[0][1], line[-1][1]
    if is_timestampt(text1):
        current_time = is_timestampt(text1)
        print(current_time)
        break
    elif is_timestampt(text2):
        current_time = is_timestampt(text2)
        print(current_time)
        break

it goes here
49:55


In [127]:
[line[0] for line in lines]

[([[95, 15], [129, 15], [129, 33], [95, 33]], 'CHE', 0.9995971751298354),
 ([[99, 39], [139, 39], [139, 55], [99, 55]], '49.55', 0.8905346903352951)]

In [ ]:
from datetime import timedelta

def convert_to_timedelta(time_str: str) -> timedelta:
    # Tách phút và giây từ chuỗi "mm:ss"
    minutes, seconds = map(int, time_str.split(':'))
    
    # Tạo đối tượng timedelta từ phút và giây
    return timedelta(minutes=minutes, seconds=seconds)

def timestamp2frames(timestamp: str, fps: str = 2):
    _, seconds = map(int, timestamp.split(':'))
    back_frames = fps * seconds
    return back_frames


# Ví dụ sử dụng
time_str1 = "30:55"  # 45 phút 55 giây
time_str2 = "30:55"  # 30 phút 20 giây

# Chuyển đổi các chuỗi thành timedelta
time1 = convert_to_timedelta(time_str1)
time2 = convert_to_timedelta(time_str2)

# Trừ thời gian
difference = time1 - time2

# In ra kết quả
print(f"Time 1: {time1}")
print(f"Time 2: {time2}")
print(f"Difference: {difference}")


Time 1: 0:30:55
Time 2: 0:30:55
Difference: 0:00:00
